### **Libraries**

In [1]:
import os
import openai
from openai import OpenAI

In [10]:
import pathlib
import textwrap

import openai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") # OpenAI API key

#### **Define a client**

In [3]:
# define an instance of OpenAI model
client = OpenAI(api_key=OPENAI_API_KEY)

#### **Audio transcription**

In [4]:
audio_file= open("audio.mp3", "rb")

In [5]:
speech_to_text = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file,
)

In [17]:
to_markdown(speech_to_text.text)

> There will be a meeting on the 21st of July at 5pm and the meeting will be about the capstone project.

### **Audio to file**

### **Function calling**

In [8]:
system_prompt = "You are an assistant that creates a well crafted schedule from text."

In [26]:
schedule_tool = [
    {
        'type': 'function',
        'function': {
            'name': 'schedule_events',
            'parameters': {
                'type': 'object',
                'properties': {
                    'date': {
                        'type': 'string',
                        'description': 'The date of the event in the format YYYY-MM-DD'
                    },
                    'start_time': {
                        'type': 'string',
                        'description': 'The time of the event starts'
                    },
                    'end_time': {
                        'type': 'string',
                        'description': 'The time of the event ends'
                    },
                    'event': {
                        'type': 'string',
                        'description': 'The name of the event'
                    },
                    'location': {
                        'type': 'string',
                        'description': 'The location of the event'
                    },
                    'description': {
                        'type': 'string',
                        'description': 'A description of the event'
                    },
                    'reminder': {
                        'type': 'string',
                        'description': 'The time before the event to send a reminder'
                    }
                }
            },
            'required': ['date', 'start_time', 'end_time', 'event'],
        },
        'instruction': system_prompt
    }
]

#### **Calendar file creation**

In [23]:
user_prompt = f"From the text {speech_to_text.text} create a calendar-like structure."

calendar_file = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {'role': 'user', 'content': user_prompt}
    ],
    tools=schedule_tool,
    tool_choice='auto',
    temperature=0.3
)

In [24]:
print(calendar_file)

ChatCompletion(id='chatcmpl-9gDZGer7iTmyockBISV0IoK2p6CvQ', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_kcaRAcxSk6yQVOuKJNbQBGPz', function=Function(arguments='{"date":"2023-07-21","time":"17:00","event":"Meeting on Capstone Project","description":"A meeting about the capstone project"}', name='schedule_events'), type='function')]))], created=1719848958, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_4008e3b719', usage=CompletionUsage(completion_tokens=43, prompt_tokens=147, total_tokens=190))


In [25]:
calendar_file.choices[0].message.content